## MTH9878 Interest Rate Models HW1
* Group X
* Author: Pan, Hongchao & Zhang, Chendi
* Kernel version: Python 2.7
* Packages: 
* Data: 
* Notes:

In [1]:
# load packages and features from version 3.x
from __future__ import absolute_import, print_function, division
import pandas as pd
import numpy as np
import math

### Answer of Q1

In [2]:
# Define a function to calculate the faction of the date differnence
def date_fraction(t1,t2):
    # Assume input t1 and t2 are calendar dates with month notation
    # Use the date convention: actual/12
    return (t2-t1)/12  

In [3]:
# Define a funtion to compute the integration by using simpson's rule
# a: left endpoint; b: right endpoint; n: number of partition intervals
# f: instantaneous rate function, f(s)
def simpson_rule(a,b,f,n):
    h= (b-a)/n
    result=(f(a) + f(b))/6

    for i in range(1, n):
        result += f(a + i*h)/3
    for i in range(1, n+1):
        result += 2*f(a + (i-0.5)*h)/3

    return result*h

In [4]:
def approx_val_tol(f_method,a,b,f,n,tol):
    """
    # f_method: integration method: mid_point/trape/simpson method; simpson's rule here
    # tol=5e-7
    # n: number of partition intervals
    # f: instantaneous rate function, f(s)
    # a: left endpoint, t here; b: right endpoint, T here;
    """
    
    result_old=f_method(a,b,f,n)
    n=2*n
    result_new=f_method(a,b,f,n)

    while(abs(result_old-result_new)>tol):
        result_old=result_new
        n=2*n
        result_new=f_method(a,b,f,n)

    return result_new


In [5]:
# Define the interest rate function
# the constant interest rate
def f_const(x):
    # example from given spreedsheet
    return 0.55/100   

# instantaneous rate, not the case in this homework
def f_instant(x):
    # Arbitrary example
    return math.exp(-x^2)

In [6]:
# Define a function to compute the discount factor
def dist(t,T,f,flag):
    """
    # t: starting date
    # T: maturity
    # f: interest rate
    # flag: indicator of f is constant rate or instantaneous rate
    """
    # Assume input t and T are calendar dates with month notation
    
    # Get the day fraction
    day_frac=date_fraction(t,T)
    if(flag=='c'):
        # constant interest rate
        return math.exp(-f*day_frac)
    if(flag=='i'):
        # instantaneous rate
        # starting intervals: 4
        # tolerance: 5e-7
        # Since t and T are calendar date, convert them to get day fraction period,
        # which is equivalent to 0 to day_frac
        return math.exp(-approx_val_tol(simpson_rule,0,day_frac,f,4,5e-7))
    else:
        print("The inputs of are not well defined.")
        return -2

### Answer of Q2

In [7]:
# define a function to calculate the LIBOR forward rate
# reference formula: equation (12) in lecture 1
def LFR(t,S,T,I,flag):
    """
    # t: spot/observation date
    # S: settlement date
    # T: maturity
    # I: instantaneous forward rate
    # T-S: underlying tenor
    # flag: indicator of I is constant or function
    # Assume input t and T are calendar dates with month notation
    """
    
    # get the day fraction
    day_frac=date_fraction(S,T)
   
    if(flag=='c'):
        # constant interest rate
        return (math.exp(I*day_frac)-1)/day_frac  # equation (12) in lecture 1
    if(flag=='i'):
        # instantaneous rate
        # starting intervals: 4
        # tolerance: 5e-7
        # Since t and T are calendar date, convert them to get day fraction period,
        # which is equivalent to 0 to day_frac
        return (math.exp(approx_val_tol(simpson_rule,0,day_frac,I,4,5e-7))-1)/day_frac
    else:
        print("The inputs are not well defined.")
        return -2
    

### Answer of Q3

In [10]:
# define a function to compute the cash flow date and value
def calculate_flow(coupon, frequency, T0,maturity):
    """
    :param coupon: in dollars
    :param frequency: is an int
    :param T0: obeservation date, in months
    :param maturity: in months
    :return: intervals scaled by years
    "This functon calculates cash flow dates and values."
    """
    
    flow_date=range(maturity,T0,int(-12/frequency))
    flow_date.reverse()
    flow_date=[t/12 for t in flow_date]   # Convert the date

    flow_value=[coupon/frequency for i in flow_date[:-1]]+[coupon/frequency]
    # flow_date[:-1] remove the last elements in the list
    return flow_date, flow_value


In [16]:
# Define a funtion to compute the spot/forward swap rate
# Reference: break-even swap rate, equation (19) in lecture note 1
def BE_swap_rate(coupon_f,frequency_f,f_f,flag_f,
                 coupon_L,frequency_L,f_L,flag_L,
                 t,T0,maturity):
    """
    _f: parameters in fix leg
    _L: parameters in LIBOR
    :param coupon: in dollars
    :param frequency: is an int
    :param t: spot date, in months, usually 0
    :param T0: obeservation date, in months
    :param maturity: in months
    :param f: interest rate
    :param flag: indicator of interest rate is constant or function
    """
    
    # Compute the annuity
    flow_date_fix,flow_value_fix=calculate_flow(coupon_f,frequency_f,T0,maturity)
    # flow_date is alpha_j in the equation (16)
    # flow_value is future cash flows
    # compute the discount factors
    df=[]
    df_append=df.append  # avoid .operation to speed up the computation
    
    for i in range(len(flow_date_fix)):
        df_append(dist(t,flow_date_fix[i],f_f,flag_f))
    
    annuity_t=sum(a*b for a,b in zip(flow_date,df))
    
    # Compute the payment of floating leg
    # Compute the flow date
    flow_date_L, flow_cash_L=calculate_flow(coupon_L,frequency_L,T0,maturity)
    # flow_date is delta_j in the equation (17)
    # flow_value is future cash flows
    # compute the discount factors
    df_L=[]
    df_L_append=df_L.append  # avoid .operation to speed up the computation
    
    for i in range(len(flow_date_L)):
        df_L_append(dist(t,flow_date_L[i],f_L,flag_L))
    
    # compute the Lj in equation (17)
    Lj=[]
    Lj_append=Lj.append
    
    for i in range(len(flow_date_L)-1):
        LJ_append(LFR(0,flow_date_L[i],flow_date_L[i+1],f_L,flag_L))
    
    P_float=sum([x*y*z for x,y,z in zip(flow_date_L,Lj,df_L)])

    return P_float/annuity_t   # break-even swap rate
    
    

### Answer of Q4